In [1]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile
import shutil

In [2]:
#from pandas import HDFStore
#hdf = HDFStore('C:/Users/nazanin/Downloads/custom_mpo_06197001_model_data.h5', mode='r')
#persons2018= hdf.get('/persons')

In [3]:
filename = "C:/Users/nazanin/Downloads/custom_mpo_06197001_model_data.h5"
persons2018 = pd.read_hdf(filename,'persons')

In [4]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [5]:
filename = "C:/Users/nazanin/Downloads/custom_mpo_06197001_model_data.h5"
households2018 = pd.read_hdf(filename,'households')

In [6]:
# Merge households and persons 
persons2018 = persons2018.sort_values(by=['household_id']).reset_index()
households2018 = households2018.sort_values(by=['household_id']).reset_index()
hhpersons2018 = pd.merge(left=persons2018, right=households2018, how='left', on='household_id')

#### PUMS

In [7]:
person_5_year_2010 = pd.read_csv('C:/Users/nazanin/Downloads/csv_pca/ss13pca.csv')

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_10256\1923606266.py:1: DtypeWarning: Columns (107,108,109) have mixed types. Specify dtype option on import or set low_memory=False.
  person_5_year_2010 = pd.read_csv('C:/Users/nazanin/Downloads/csv_pca/ss13pca.csv')


In [8]:
housing_unit_5_year_2010 = pd.read_csv('C:/Users/nazanin/Downloads/csv_hca/ss13hca.csv')

In [9]:
# Merge households and persons 
person_5_year_2010 = person_5_year_2010.sort_values(by=['SERIALNO']).reset_index()
housing_unit_5_year_2010 = housing_unit_5_year_2010.sort_values(by=['SERIALNO']).reset_index()
hhpersons_PUMS2010 = pd.merge(left=person_5_year_2010, right=housing_unit_5_year_2010, how='left', on='SERIALNO')

In [10]:
hhpersons_PUMS2010 = hhpersons_PUMS2010.sort_values(by=['SERIALNO', 'SPORDER']).reset_index(drop=True)
hhpersons2018 = hhpersons2018.sort_values(by=['serialno', 'member_id']).reset_index(drop=True)
hhpersons2018_dis = pd.merge(hhpersons2018, hhpersons_PUMS2010[['SERIALNO', 'SPORDER', 'AGEP', 'SEX', 'DIS', 'HINCP', 'VEH', 'JWMNP', 'JWRIP', 'JWTR', 'RAC1P', 'RAC2P05']], how='left',left_on = ['serialno', 'member_id'] , right_on=['SERIALNO', 'SPORDER'])

In [11]:
columns_persons = persons2018.columns.tolist()

In [12]:
columns_persons.append('DIS')

In [13]:
persons = hhpersons2018_dis.loc[:, hhpersons2018_dis.columns.isin(columns_persons)]  

In [14]:
persons['in_wheelchair'] = persons['DIS']==1

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_10256\3772360376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  persons['in_wheelchair'] = persons['DIS']==1


In [15]:
persons.head()

,person_id,earning,worker,student,hispanic.1,person_sex,member_id,sex,household_id,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,DIS,in_wheelchair
0,2308939,100000.0,1,0,0,male,1,1,2422236,40.0,3,35 to 60,21.0,-1,1,white,0,58,1376,no,0,0,2,False
1,522659,100000.0,1,0,0,male,1,1,2422949,40.0,3,35 to 60,21.0,-1,1,white,0,58,1361,no,0,0,2,False
2,3017711,100000.0,1,0,0,male,1,1,2425565,40.0,3,35 to 60,21.0,-1,1,white,0,58,1359,no,0,0,2,False
3,3638104,100000.0,1,0,0,male,1,1,2425757,40.0,3,35 to 60,21.0,-1,1,white,0,58,1395,no,0,0,2,False
4,3032345,100000.0,1,0,0,male,1,1,2426936,40.0,3,35 to 60,21.0,-1,1,white,0,58,1365,no,0,0,2,False


In [16]:
persons.to_hdf(filename, key = 'persons')

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_10256\3028467451.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['person_sex', 'person_age', 'school_zone_id', 'race', 'work_zone_id',
       'p_hispanic'],
      dtype='object')]

  persons.to_hdf(filename, key = 'persons')


In [17]:
#Checking
persons_new = pd.read_hdf(filename,'persons')

In [31]:
persons_test = persons_new.sort_values("household_id").reset_index(drop=True)

In [32]:
persons_test.head()

,person_id,earning,worker,student,hispanic.1,person_sex,member_id,sex,household_id,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,DIS,in_wheelchair
0,0,116000.0,1,0,0,female,1,2,0,40.0,5,above 60,22.0,-1,6,asian,0,61,387,no,0,0,2,False
1,152718,110000.0,1,0,0,male,1,1,1,55.0,5,35 to 60,22.0,-1,2,black,0,51,401,no,0,0,2,False
2,210442,0.0,0,0,0,female,1,2,2,0.0,3,above 60,23.0,-1,1,white,0,64,-1,no,0,0,2,False
3,257710,100000.0,1,0,0,female,1,2,3,50.0,5,35 to 60,22.0,-1,1,white,0,53,412,no,0,0,2,False
4,325576,121000.0,1,0,0,male,1,1,4,40.0,3,above 60,19.0,-1,1,white,0,62,1141,no,0,0,2,False


In [19]:
persons2018.head()

,person_id,earning,worker,student,hispanic.1,person_sex,member_id,sex,household_id,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic
0,0,116000.0,1,0,0,female,1,2,0,40.0,5,above 60,22.0,-1,6,asian,0,61,387,no,0,0
1,152718,110000.0,1,0,0,male,1,1,1,55.0,5,35 to 60,22.0,-1,2,black,0,51,401,no,0,0
2,210442,0.0,0,0,0,female,1,2,2,0.0,3,above 60,23.0,-1,1,white,0,64,-1,no,0,0
3,257710,100000.0,1,0,0,female,1,2,3,50.0,5,35 to 60,22.0,-1,1,white,0,53,412,no,0,0
4,325576,121000.0,1,0,0,male,1,1,4,40.0,3,above 60,19.0,-1,1,white,0,62,1141,no,0,0


In [17]:
#Assuming you do not know your hierarchy structure in advance, you can use a recursive algorithm to yield, via an iterator, full paths to all available datasets in the form group1/group2/.../dataset.
def traverse_datasets(hdf_file):

    def h5py_dataset_iterator(g, prefix=''):
        for key in g.keys():
            item = g[key]
            path = f'{prefix}/{key}'
            if isinstance(item, h5py.Dataset): # test for dataset
                yield (path, item)
            elif isinstance(item, h5py.Group): # test for group (go down)
                yield from h5py_dataset_iterator(item, path)

    for path, _ in h5py_dataset_iterator(hdf_file):
        yield path

In [18]:
with h5py.File(filename, 'r') as f:
    for dset in traverse_datasets(f):
        print('Path:', dset)
        print('Shape:', f[dset].shape)
        print('Data type:', f[dset].dtype)

Path: /blocks/axis0
Shape: (15,)
Data type: |S25
Path: /blocks/axis1
Shape: (108469,)
Data type: |S15
Path: /blocks/block0_items
Shape: (8,)
Data type: |S14
Path: /blocks/block0_values
Shape: (1,)
Data type: object
Path: /blocks/block1_items
Shape: (5,)
Data type: |S25
Path: /blocks/block1_values
Shape: (108469, 5)
Data type: float64
Path: /blocks/block2_items
Shape: (2,)
Data type: |S18
Path: /blocks/block2_values
Shape: (108469, 2)
Data type: int64
Path: /counties/axis0
Shape: (2,)
Data type: |S9
Path: /counties/axis1
Shape: (9,)
Data type: int64
Path: /counties/block0_items
Shape: (1,)
Data type: |S9
Path: /counties/block0_values
Shape: (1,)
Data type: object
Path: /counties/block1_items
Shape: (1,)
Data type: |S8
Path: /counties/block1_values
Shape: (9, 1)
Data type: float64
Path: /ect/axis0
Shape: (1,)
Data type: |S20
Path: /ect/axis1
Shape: (41,)
Data type: int64
Path: /ect/block0_items
Shape: (1,)
Data type: |S20
Path: /ect/block0_values
Shape: (41, 1)
Data type: float64
Path: /

In [20]:
f = h5py.File(filename, 'r')
list(f.keys())

['blocks',
 'counties',
 'ect',
 'edges',
 'hct',
 'household_targets_acs',
 'household_validation_acs',
 'households',
 'job_county_targets_BEA',
 'job_county_validation_BEA',
 'job_flows',
 'job_targets',
 'job_validation',
 'jobs',
 'nodes',
 'persons',
 'price_targets',
 'price_validation',
 'residential_units',
 'taz_zone_geoms',
 'travel_data',
 'unit_county_targets_PEP',
 'unit_county_validation_PEP',
 'unit_targets',
 'unit_validation',
 'validation_ect',
 'validation_hct',
 'values']

In [21]:
list(f['persons'].keys())

['axis0',
 'axis1',
 'block0_items',
 'block0_values',
 'block1_items',
 'block1_values',
 'block2_items',
 'block2_values']

In [17]:
#list(f['households'].keys())

In [18]:
#dset_jobs = f['/jobs/axis0']

In [19]:
#list(dset_jobs[:])

In [26]:
dset_persons = f['/persons/block0_items']

In [27]:
list(dset_persons[:])

[b'earning', b'hours', b'edu']

In [24]:
#dset1 = f['/persons/block0_items']

In [25]:
#dset2 = f['/persons/block0_values']

In [26]:
#dset3 = f['/persons/block1_items']

In [27]:
#dset4 = f['/persons/block1_values']

In [28]:
#dset5 = f['/persons/block2_items']

In [29]:
#dset6 = f['/persons/block2_values']